In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [2]:
!kaggle datasets download -d rahilmehtaucoe2784/indian-currency

 76% 65.0M/85.7M [00:01<00:00, 28.7MB/s]
100% 85.7M/85.7M [00:01<00:00, 77.0MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools



     |████████████████████████████████| 616 kB 13.6 MB/s 
     |████████████████████████████████| 596 kB 70.6 MB/s 
     |████████████████████████████████| 1.2 MB 60.5 MB/s 
     |████████████████████████████████| 6.4 MB 54.1 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 1.1 MB 68.8 MB/s 
     |████████████████████████████████| 3.4 MB 66.0 MB/s 
     |████████████████████████████████| 840 kB 63.8 MB/s 
     |████████████████████████████████| 87 kB 7.0 MB/s 
     |████████████████████████████████| 120 kB 51.0 MB/s 
     |████████████████████████████████| 234 kB 65.7 MB/s 
     |████████████████████████████████| 1.1 MB 36.7 MB/s 
     |████████████████████████████████| 25.3 MB 1.3 MB/s 
     |████████████████████████████████| 99 kB 11.0 MB/s 
     |████████████████████████████████| 352 kB 82.1 MB/s 
     |████████████████████████████████| 47.7 MB 1.2 MB/s 
     |████████████████████████████████| 462 kB 64.7 MB/s 
     |█████████████

In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import time

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

In [6]:
train_image_dir = '/content/final_currency_dataset/train images'
train_annotation_dir = '/content/final_currency_dataset/train anotation'
validate_image_dir = '/content/final_currency_dataset/validate Images'
validate_anaotation_dir = '/content/final_currency_dataset/validate anotation '
test_image_dir = '/content/final_currency_dataset/test image'
test_anotation_dir = '/content/final_currency_dataset/test anotation'

In [7]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"10", 2: "20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [8]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [9]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_anotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [10]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

2383
671
345


In [13]:
spec = model_spec.get('efficientdet_lite4')

In [18]:
start = time.time()
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=6, 
                               train_whole_model=True, 
                               validation_data=validate_dir)
end = time.time()

Epoch 1/10
397/397 [==============================] - 595s 1s/step - det_loss: 0.9694 - cls_loss: 0.7847 - box_loss: 0.0037 - reg_l2_loss: 0.1090 - loss: 1.0784 - learning_rate: 0.0078 - gradient_norm: 3.6196 - val_det_loss: 0.5294 - val_cls_loss: 0.4751 - val_box_loss: 0.0011 - val_reg_l2_loss: 0.1094 - val_loss: 0.6388
Epoch 2/10
397/397 [==============================] - 539s 1s/step - det_loss: 0.5936 - cls_loss: 0.5026 - box_loss: 0.0018 - reg_l2_loss: 0.1096 - loss: 0.7032 - learning_rate: 0.0070 - gradient_norm: 3.3884 - val_det_loss: 0.4446 - val_cls_loss: 0.3934 - val_box_loss: 0.0010 - val_reg_l2_loss: 0.1097 - val_loss: 0.5543
Epoch 3/10
397/397 [==============================] - 541s 1s/step - det_loss: 0.5118 - cls_loss: 0.4365 - box_loss: 0.0015 - reg_l2_loss: 0.1099 - loss: 0.6216 - learning_rate: 0.0061 - gradient_norm: 3.6082 - val_det_loss: 0.3776 - val_cls_loss: 0.3334 - val_box_loss: 8.8301e-04 - val_reg_l2_loss: 0.1100 - val_loss: 0.4876
Epoch 4/10
397/397 [=======

In [19]:
print("The time of execution of training is :", end-start)

The time of execution of training is : 5564.068696022034


In [23]:
start = time.time()
ap = model.evaluate(test_dir,batch_size=5)
print(ap)
end = time.time()


69/69 [==============================] - 22s 251ms/step

{'AP': 0.78369546, 'AP50': 0.89814377, 'AP75': 0.8494696, 'APs': -1.0, 'APm': 0.5592966, 'APl': 0.7969842, 'ARmax1': 0.74215245, 'ARmax10': 0.8527381, 'ARmax100': 0.8586781, 'ARs': -1.0, 'ARm': 0.63166666, 'ARl': 0.87120247, 'AP_/10': 0.68068445, 'AP_/20': 0.80811995, 'AP_/50': 0.7562806, 'AP_/100': 0.86769056, 'AP_/200': 0.84757006, 'AP_/500': 0.7874181, 'AP_/2000': 0.7381046}


In [24]:
print("The time of execution of evaluation is  :", end-start)

The time of execution of evaluation is  : 24.731475353240967


In [ ]:
#model = object_detector.create(train_dir, model_spec=spec, epochs=30, batch_size=8, train_whole_model=True)

In [22]:
model.export(export_dir='/content/')